In [70]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tqdm
%matplotlib inline

ImportError: No module named 'tqdm'

In [71]:
# Read files
with open("//QATLPCFS001/Users/skiran/Downloads/cornell_movie_dialogs_corpus/movie_conversations.txt", 'r') as conv_file:
    conv_raw = pd.read_csv(conv_file, sep=" \+\+\+\$\+\+\+ ", header = None, engine = 'python')
    conv_raw.columns = ['person1', 'person2', 'movie', 'conv_seq']

with open("//QATLPCFS001/Users/skiran/Downloads/cornell_movie_dialogs_corpus/movie_lines.txt", 'r') as conv_file:
    conv_lines = pd.read_csv(conv_file, sep=" \+\+\+\$\+\+\+ ", header = None, engine = 'python')
    conv_lines.columns = ['line_num', 'person', 'movie', 'person_name', 'line']

In [72]:
conv_raw.head(5)

,person1,person2,movie,conv_seq
0,u0,u2,m0,"['L194', 'L195', 'L196', 'L197']"
1,u0,u2,m0,"['L198', 'L199']"
2,u0,u2,m0,"['L200', 'L201', 'L202', 'L203']"
3,u0,u2,m0,"['L204', 'L205', 'L206']"
4,u0,u2,m0,"['L207', 'L208']"


In [73]:
conv_lines.head(5)

,line_num,person,movie,person_name,line
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [74]:
# extract dialogue sequnce as an array
from ast import literal_eval
conv_raw['conv_seq'] = conv_raw['conv_seq'].apply(lambda x: [line.replace('L', '') for line in literal_eval(x)])
conv_raw['first_line'] = conv_raw['conv_seq'].apply(lambda x: x[0])
conv_raw['last_line'] = conv_raw['conv_seq'].apply(lambda x: x[-1])

In [97]:
conv_raw.sort_values(by = ['movie', 'person1', 'person2', 'first_line']).reset_index(drop = True)
conv_raw['unique_ppm'] = conv_raw['person1'] + conv_raw['person2'] + conv_raw['movie']
conv_raw['unique_row_id'] = conv_raw['unique_ppm'] + "_" + conv_raw['last_line']
conv_raw.head(10)

,person1,person2,movie,conv_seq,first_line,last_line,unique_ppm,unique_row_id
0,u0,u2,m0,"[194, 195, 196, 197]",194,197,u0u2m0,u0u2m0_197
1,u0,u2,m0,"[198, 199]",198,199,u0u2m0,u0u2m0_199
2,u0,u2,m0,"[200, 201, 202, 203]",200,203,u0u2m0,u0u2m0_203
3,u0,u2,m0,"[204, 205, 206]",204,206,u0u2m0,u0u2m0_206
4,u0,u2,m0,"[207, 208]",207,208,u0u2m0,u0u2m0_208
5,u0,u2,m0,"[271, 272, 273, 274, 275]",271,275,u0u2m0,u0u2m0_275
6,u0,u2,m0,"[276, 277]",276,277,u0u2m0,u0u2m0_277
7,u0,u2,m0,"[280, 281]",280,281,u0u2m0,u0u2m0_281
8,u0,u2,m0,"[363, 364]",363,364,u0u2m0,u0u2m0_364
9,u0,u2,m0,"[365, 366]",365,366,u0u2m0,u0u2m0_366


In [142]:
from functools import reduce
def merge_lists(series):
    if len(series) > 1:
        return reduce(lambda x, y: x + y, series)
    else:
        return series[0]

g = (conv_raw['unique_ppm'] + "_" + (conv_raw['first_line'].map(int) - 1).map(str) != conv_raw.shift().fillna(method='bfill')['unique_row_id']).cumsum().rename('group')
conv_agg = conv_raw.groupby(['person1', 'person2', 'movie', g])['conv_seq'].apply(list).reset_index().drop('group',axis=1)
conv_agg['conv_seq'] = conv_agg['conv_seq'].apply(lambda l: [item for sublist in l for item in sublist])

In [148]:
print(conv_agg.head(10))
print(conv_raw.shape, conv_agg.shape)

  person1 person2 movie                                           conv_seq
0      u0     u11    m0                          [179, 180, 181, 182, 183]
1      u0     u11    m0                                         [189, 190]
2      u0     u11    m0                          [517, 518, 519, 520, 521]
3      u0     u11    m0                                         [523, 524]
4      u0     u11    m0                          [536, 537, 538, 539, 540]
5      u0     u11    m0                                    [544, 545, 546]
6      u0     u11    m0                [878, 879, 880, 881, 882, 883, 884]
7      u0     u11    m0                                         [922, 923]
8      u0      u2    m0  [194, 195, 196, 197, 198, 199, 200, 201, 202, ...
9      u0      u2    m0                [271, 272, 273, 274, 275, 276, 277]
(83097, 8) (60699, 4)


In [188]:
import re
raw_sent = "   -Sai --Kiran---Vadhi's-123.. <a> Sai \"you've been, 've doin' . nothin' but\" it 's ?everythin   "
print(raw_sent)
exceptions = r"\b(it|that|there|he|she) (\'s)"
print(re.sub(exceptions, r'\1\2', raw_sent))

   -Sai --Kiran---Vadhi's-123.. <a> Sai "you've been, 've doin' . nothin' but" it 's ?everythin   
   -Sai --Kiran---Vadhi's-123.. <a> Sai "you've been, 've doin' . nothin' but" it's ?everythin   


In [189]:
import re

def clean_sentence(raw_sent):
    raw_sent = raw_sent.lower()
    # remove html tags like <i>, </u>
    raw_sent = re.sub(r'<\/*[a-z]?>', '', raw_sent)
    # special character for 'pause' (..., ---) replaced with "<pause>"
    raw_sent = re.sub(r'\s*(\.\.+|--+)\s*', ' <pause> ', raw_sent)
    # words like doin', goin' to be replaced with doing, going
    raw_sent = re.sub(r'([a-z]+)in\'', r'\1ing', raw_sent)
    # you've, there've replaced with you have, there have
    raw_sent = re.sub(r'([a-z]+)\'ve', r'\1 have', raw_sent)
    # remove double quotes
    raw_sent = re.sub(r'\"', '', raw_sent)
    # treat commas and other EOS tags as words (put space around them)
    raw_sent = re.sub(r'\s*([\.,!\?]+)\s*', r'\s\1\s', raw_sent)
    # 's to be seperated from the word with some exceptions
    raw_sent = re.sub(r'([a-z]+)(\'s)', r'\1 \2', raw_sent)
    exceptions = r"\b(it|that|there|he|she) (\'s)"
    raw_sent = re.sub(exceptions, r'\1\2', raw_sent)
    # remove leading and trailing spaces
    raw_sent = re.sub(r'^\s*|\s*$', '', raw_sent)
    # add <eos> tag at the end of sentence
    raw_sent = raw_sent + " <eos>"
    return(raw_sent)